In [4]:
!python -m pip install mlflow==1.24.1.dev0 --find-links=https://mlflow-snapshots.s3-us-west-2.amazonaws.com/mlflow-1.24.1.dev0-0.0825716d-py2.py3-none-any.whl

Looking in links: https://mlflow-snapshots.s3-us-west-2.amazonaws.com/mlflow-1.24.1.dev0-0.0825716d-py2.py3-none-any.whl
  Using cached https://mlflow-snapshots.s3-us-west-2.amazonaws.com/mlflow-1.24.1.dev0-0.0825716d-py2.py3-none-any.whl (16.5 MB)
  Attempting uninstall: mlflow
    Found existing installation: mlflow 1.22.0
    Uninstalling mlflow-1.22.0:
      Successfully uninstalled mlflow-1.22.0
You should consider upgrading via the '/home/akinwilson/.pyenv/versions/3.8.2/bin/python -m pip install --upgrade pip' command.


In [1]:
import mlflow
import os
import requests 
from pprint import pprint 
from dotenv import load_dotenv

In [2]:
import os
load_dotenv()
DEFAULT_ARTIFACTS_URI = os.environ['MLFLOW_TRACKING_URI']
print(DEFAULT_ARTIFACTS_URI)

http://mlops-alb-development-202947496.eu-west-2.elb.amazonaws.com/mlops/


In [3]:
from mlflow.store.artifact.mlflow_artifacts_repo import MlflowArtifactsRepository
MlflowArtifactsRepository(DEFAULT_ARTIFACTS_URI).list_artifacts()

[]

In [4]:
print("Tracking server uri before loading env vars:")
print(mlflow.tracking.get_tracking_uri())
load_dotenv()
print("Tracking server uri after loading env vars:")
print(mlflow.tracking.get_tracking_uri())

Tracking server uri before loading env vars:
http://mlops-alb-development-202947496.eu-west-2.elb.amazonaws.com/mlops/
Tracking server uri after loading env vars:
http://mlops-alb-development-202947496.eu-west-2.elb.amazonaws.com/mlops/


In [5]:
!mlflow experiments create  -n nlp

Created experiment 'nlp' with id 1


How do I see if it is authenticated or not?

In [6]:
mlflow.create_experiment("NLPv2")

'2'

In [7]:
import warnings
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

def training():
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file from the URL
    csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    )
    try:
        data = pd.read_csv(csv_url, sep=";")
    except Exception as e:
        logger.exception(
            "Unable to download training & test CSV, check your internet connection. Error: %s", e
        )

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    alpha =  0.5
    l1_ratio = 0.5
    
    exp_id = mlflow.create_experiment("NLPv5")
    
    print(mlflow.get_experiment(exp_id))
    
    mlflow.start_run(experiment_id=exp_id)
    
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)
    predicted_qualities = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

    # Model registry does not work with file store
    if tracking_url_type_store != "file":

        # Register the model
        # There are other ways to use the Model Registry, which depends on the use case,
        # please refer to the doc for more information:
        # https://mlflow.org/docs/latest/model-registry.html#api-workflow
        mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticnetWineModel")
    else:
        mlflow.sklearn.log_model(lr, "model")
        
    mlflow.end_run()

In [8]:
training()

<Experiment: artifact_location='mlflow-artifacts:/3', experiment_id='3', lifecycle_stage='active', name='NLPv5', tags={}>
Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.7931640229276851
  MAE: 0.6271946374319586
  R2: 0.10862644997792614


Successfully registered model 'ElasticnetWineModel'.
2022/03/19 07:29:46 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetWineModel, version 1
Created version '1' of model 'ElasticnetWineModel'.


In [ ]:
mlflow.active_run().__dict__
mlflow.end_run()